<a href="https://colab.research.google.com/github/ryo42164/kaggle/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer,KNNImputer,IterativeImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedKFold,StratifiedGroupKFold,cross_val_score,RepeatedStratifiedKFold
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [ ]:
sub = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NFL Draft Prediction/data/sample_submission.csv")
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NFL Draft Prediction/data/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NFL Draft Prediction/data/test.csv")

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")

In [ ]:
test_or_sub = 'test'

In [ ]:
X = train.drop(['Id','Drafted'],axis=1)
y = train['Drafted']
X_test = test.drop('Id',axis=1)

In [ ]:
high_missing_num = list(train.isnull().sum()[train.isnull().sum() > 500].index)

group_cols = ['Year','Position_Type']
target_cols = ['Sprint_40yd', 'Vertical_Jump', 'Bench_Press_Reps','Broad_Jump', 'Agility_3cone', 'Shuttle', 'Height', 'Weight']
new_cols = ['SpeedScore','ExplosivePower','AgilityCombo','StrengthIndex','PowerToSize']
CV = RepeatedStratifiedKFold(n_splits=5,n_repeats=2,random_state=1)
group = None
#group =train['Year']
#CV = StratifiedGroupKFold(n_splits=5,shuffle=True,random_state=1)
global_mean = y.mean()

In [ ]:
high_missing_num

['Vertical_Jump', 'Bench_Press_Reps', 'Broad_Jump', 'Agility_3cone', 'Shuttle']

In [ ]:
def target_encode_smooth(X,y,X_test,col,global_mean=global_mean,cv=CV,k=10,group=group):
  oof_te = np.zeros(len(X))
  if len(col) == 1:
    single= True
    col = col[0]
  else:
    single = False

  for trn_idx,val_idx in cv.split(X,y,groups=group):
    X_trn,y_trn = X.iloc[trn_idx],y.iloc[trn_idx]
    df_trn = X_trn.copy()
    df_trn['Drafted'] = y_trn

    stats = df_trn.groupby(col)['Drafted'].agg(['mean','count'])
    alpha = stats['count']/(stats['count']+k)
    smoothed = alpha * stats['mean'] + (1 - alpha) *global_mean

    if single:
      oof_te[val_idx] = X.iloc[val_idx][col].map(smoothed)
    else:
      idx = list(zip(*(X.iloc[val_idx][c]for c in col)))
      oof_te[val_idx] = pd.Series(idx).map(smoothed)
  oof_te = np.where(np.isnan(oof_te),global_mean,oof_te)

  df_full = X.copy()
  df_full['Drafted'] = y
  stats_full = df_full.groupby(col)['Drafted'].agg(['mean','count'])
  alpha_full = stats_full['count']/(stats_full['count']+k)
  smoothed_full = alpha_full * stats_full['mean'] + (1-alpha_full)*global_mean
  if single:
    X[col+'_te_smoothed'] = oof_te
    X_test[col+'_te_smoothed'] = X_test[col].map(smoothed_full).fillna(global_mean)
  else:
    key = '_'.join(col)
    X[key + '_te_smoothed'] = oof_te

    idx_test = list(zip(*(X_test[c] for c in col)))
    X_test[key+'_te_smoothed'] = pd.Series(idx_test).map(smoothed_full).fillna(global_mean)
  return X,X_test

In [ ]:

FEATURE_FLAGS = {
    'School_counts': True,
    'Missing_columns': True,
    'Age_missing':True,
    'School_te':True,
    'Position_te':False,
    'Position_smoothed_te':False,
    'Position_Type_smoothed_te':True,
    'PosStats':True,
    'BMI':True,
    'SpeedScore': True,
    'ExplosivePower': True,
    'AgilityCombo': True,
    'StrengthIndex': True,
    'PowerToSize': True,
    'SchoolPos_te':False,
    'Year_smoothed_te':True,
    'Year_counts':True,
    'Year_PositionType_te':True,
    'missing_rate_by_year':True,
    'Year_PositionType_count':True,
    'missing_cluster':True,
    'missing_PT':True
}

if FEATURE_FLAGS['School_counts']:
  all_school_counts = pd.concat([train['School'],test['School']]).value_counts()
  oof = np.zeros(len(X))
  for tr_idx, val_idx in CV.split(X, y,groups=group):
    X_trn, y_trn = X.iloc[tr_idx], y[tr_idx]
    school_counts = X_trn['School'].value_counts()
    oof[val_idx] = X.iloc[val_idx]['School'].map(school_counts).fillna(0)
  X['School_count'] = oof
  X_test['School_count'] = test['School'].map(all_school_counts).fillna(0)

if FEATURE_FLAGS['Missing_columns']:
  for col in high_missing_num+['Sprint_40yd']:
    if not col == 'Age':
      X[col+'_missing'] = X[col].isnull().astype(int)
      X_test[col+'_missing'] = X_test[col].isnull().astype(int)

if FEATURE_FLAGS['Age_missing']:
  X['Age_missing'] = X['Age'].isnull().astype(int)
  X_test['Age_missing'] = X_test['Age'].isnull().astype(int)

if FEATURE_FLAGS['School_te']:
  X,X_test = target_encode_smooth(X,y,X_test,['School'])

if FEATURE_FLAGS['Position_te']:
  oof_te = np.zeros(len(X))

  for trn_idx,val_idx in CV.split(X,y):
    X_trn,y_trn = X.iloc[trn_idx],y.iloc[trn_idx]
    df_trn = X_trn.copy()
    df_trn['Drafted'] = y_trn

    te_map = df_trn.groupby('Position')['Drafted'].mean()
    oof_te[val_idx] = X.iloc[val_idx]['Position'].map(te_map)
  oof_te  = np.where(np.isnan(oof_te),global_mean,oof_te)
  df_full = X.copy()
  df_full['Drafted'] = y
  map_te_all = df_full.groupby('Position')['Drafted'].mean()
  X['Position_te'] = oof_te
  X_test['Position_te'] = X_test['Position'].map(map_te_all).fillna(global_mean)

if FEATURE_FLAGS['Position_smoothed_te']:
  X,X_test = target_encode_smooth(X,y,X_test,['Position'])

if FEATURE_FLAGS['Position_Type_smoothed_te']:
  X,X_test = target_encode_smooth(X,y,X_test,['Position_Type'])

if FEATURE_FLAGS['BMI']:
  X['BMI'] = X['Weight'] / (X['Height'] ** 2)
  X_test['BMI'] = X_test['Weight'] /(X_test['Height']**2)

# SpeedScore: 体重考慮の40ydスコア（有名な指標）
if FEATURE_FLAGS.get('SpeedScore'):
    X['SpeedScore'] = (X['Weight'] * 200) / (X['Sprint_40yd'] ** 4)
    X_test['SpeedScore'] = (X_test['Weight'] * 200) / (X_test['Sprint_40yd'] ** 4)

# ExplosivePower: 跳躍系を掛け合わせる
if FEATURE_FLAGS.get('ExplosivePower'):
    X['ExplosivePower'] = X['Vertical_Jump'] * X['Broad_Jump']
    X_test['ExplosivePower'] = X_test['Vertical_Jump'] * X_test['Broad_Jump']

# AgilityCombo: シャトルと3coneの平均
if FEATURE_FLAGS.get('AgilityCombo'):
    X['AgilityCombo'] = (X['Agility_3cone'] + X['Shuttle']) / 2
    X_test['AgilityCombo'] = (X_test['Agility_3cone'] + X_test['Shuttle']) / 2

# StrengthIndex: ベンチ / 体重
if FEATURE_FLAGS.get('StrengthIndex'):
    X['StrengthIndex'] = X['Bench_Press_Reps'] / X['Weight']
    X_test['StrengthIndex'] = X_test['Bench_Press_Reps'] / X_test['Weight']

# PowerToSize: 跳躍の爆発力 ÷ 身長
if FEATURE_FLAGS.get('PowerToSize'):
    X['PowerToSize'] = (X['Vertical_Jump'] * X['Broad_Jump']) / X['Height']
    X_test['PowerToSize'] = (X_test['Vertical_Jump'] * X_test['Broad_Jump']) / X_test['Height']

if FEATURE_FLAGS['PosStats']:
  for col in target_cols + new_cols:
    grp = X.groupby(group_cols)[col]
    mean = grp.transform('mean')
    std = grp.transform('std')

    X[f'{col}_z_pos'] = (X[col]-mean)/std
    X[f'{col}_rank_pos'] = grp.rank(pct=True)

    stats = X.groupby(group_cols)[col].agg(['mean','std'])
    merged = X_test.merge(stats,how='left',left_on = group_cols,right_index=True)

    X_test[f'{col}_z_pos'] = (X_test[col]-merged['mean'])/merged['std']

    mean_rank = X[f'{col}_rank_pos'].mean()
    grp_rank = X.groupby(group_cols)[f'{col}_rank_pos'].mean()

    X_test = X_test.merge(grp_rank.rename(f'{col}_rank_pos'),how='left',left_on=group_cols,right_index=True)
    X_test[f'{col}_rank_pos'] = X_test[f'{col}_rank_pos'].fillna(mean_rank)


if FEATURE_FLAGS.get('SchoolPos_te'):
  X,X_test = target_encode_smooth(X,y,X_test,['School','Position'])

if FEATURE_FLAGS['Year_smoothed_te']:
  X,X_test = target_encode_smooth(X,y,X_test,['Year'],cv=RepeatedStratifiedKFold(n_splits=5,n_repeats=2,random_state=1),group=None)

if FEATURE_FLAGS.get('Year_counts'):
  year_counts = X['Year'].value_counts()
  X['Year_count'] = X['Year'].map(year_counts)
  all_year_counts = pd.concat([train['Year'], test['Year']]).value_counts()
  X_test['Year_count'] = X_test['Year'].map(all_year_counts).fillna(0)

if FEATURE_FLAGS.get('Year_te'):
    X, X_test = target_encode_smooth(X, y, X_test, ['Year'])

if FEATURE_FLAGS.get('Year_PositionType_te'):
    X, X_test = target_encode_smooth(X, y, X_test, ['Year','Position_Type'])

if FEATURE_FLAGS.get('missing_rate_by_year'):
  for col in high_missing_num:
    miss_rate_by_year = train.groupby('Year')[col].apply(lambda s: s.isnull().mean())
    X[col+'_missing_rate_year'] = X['Year'].map(miss_rate_by_year)
    X_test[col+'_missing_rate_year'] = X_test['Year'].map(miss_rate_by_year)

if FEATURE_FLAGS.get('Year_PositionType_count'):
    grp = X.groupby(['Year','Position_Type']).size()
    X['Year_PositionType_count'] = list(
        zip(X['Year'], X['Position_Type'])
    )
    X['Year_PositionType_count'] = X['Year_PositionType_count'].map(grp)

    all_grp = pd.concat([
        train[['Year','Position_Type']],
        test[['Year','Position_Type']]
    ]).groupby(['Year','Position_Type']).size()

    X_test['Year_PositionType_count'] = list(
        zip(X_test['Year'], X_test['Position_Type'])
    )
    X_test['Year_PositionType_count'] = X_test['Year_PositionType_count'].map(all_grp).fillna(0)

if FEATURE_FLAGS.get('missing_cluster'):
  df = pd.concat([train,test])
  missing_matrix = df[high_missing_num].isna().astype(int)
  kmeans = KMeans(n_clusters=5,random_state=1,n_init='auto')
  clusters = kmeans.fit_predict(missing_matrix)
  X['missing_cluster'] = clusters[~df['Drafted'].isnull()]
  X_test['missing_cluster'] = clusters[df['Drafted'].isnull()]
  X['missing_cluster'] = X['missing_cluster'].astype('category')
  X_test['missing_cluster'] = X_test['missing_cluster'].astype('category')

if FEATURE_FLAGS.get('missing_PT'):
  for col in high_missing_num+['Sprint_40yd']:
    for p in train['Position_Type']:
      X[f'{col}_missing_PT_{p}'] = ((X[f'{col}_missing']==1)& (X['Position_Type']== p)).astype(int)
      X_test[f'{col}_missing_PT_{p}'] = ((X_test[f'{col}_missing']==1)& (X_test['Position_Type']== p)).astype(int)

if FEATURE_FLAGS.get('missing_rate_year'):
  for col in high_missing_num + ['Sprint_40yd']:
    year_missing_rate = X.groupby('Year')[col].apply(lambda x: x.isna().mean())

    X[f'{col}_missing_rate_year'] = X['Year'].map(year_missing_rate)
    X_test[f'{col}_missing_rate_year'] = X_test['Year'].map(year_missing_rate).fillna(0)

In [ ]:
X['Year'] = X['Year'].astype('category')
X_test['Year'] = X_test['Year'].astype('category')

In [ ]:
num_proc = Pipeline(steps = [
    ('imputer',IterativeImputer(random_state=1,max_iter=10,initial_strategy='median')),
    ('scaler',StandardScaler())
])

num_proc_high = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median'))
])

cat_proc = Pipeline(steps = [
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown = 'ignore',sparse_output=False,))
])

missing_flag_proc = 'passthrough'

num_candidates = list(X.select_dtypes(exclude=['object','category']).columns)
missing_flag_cols = [c for c in X.columns if 'missing' in c]
cat_features = [c for c in X.select_dtypes(include=['object','category']).columns if c != 'School']
pos_stats = [c for c in X.columns if c.endswith('_z_pos') or c.endswith('_rank_pos')]
te_cols = [c for c in X.columns if c.endswith('_te_smoothed')]
age_missing=['Age_missing']
exclude_set = set(missing_flag_cols) | set(high_missing_num) | set(pos_stats) | set(te_cols) | set(target_cols) | set(new_cols)
num_features = [c for c in num_candidates if c not in exclude_set]

def make_preprocessor(model_name):
  """
  model_name : 'lgbm','xgb','cat'
  """
  if model_name == 'cat':
    num_proc_cat = Pipeline([
        ('imputer',SimpleImputer(strategy='median'))
    ])
    cat_features_local = cat_features + ['School']
    num_cat_features = num_features + new_cols
    num_cat_features.remove('School_count')
    pre = ColumnTransformer(
        transformers=[
            ('num',num_proc_cat,num_cat_features),
            #('num_high',num_proc_high,high_missing_num),
            #('missing_flag', 'passthrough', missing_flag_cols),
            #('pos_stats', 'passthrough', pos_stats),
            #('te_cols', 'passthrough', te_cols),
            #('age_missing','passthrough',age_missing),
            ('cat_raw', 'passthrough', cat_features_local)
        ],
        remainder='drop'
    )
    return pre
  elif model_name == 'lgbm':
    pre = ColumnTransformer(
        transformers=[
            ('num', num_proc, num_features),
            ('num_high', num_proc_high, high_missing_num),
            ('missing_flag', missing_flag_proc, missing_flag_cols),
            ('cat', cat_proc, cat_features),
            ('pos_stats', 'passthrough', pos_stats),
            ('te_cols', 'passthrough', te_cols)
        ],
        remainder='drop'
    )
    return pre
  elif model_name == 'xgb':
    pre = ColumnTransformer(
        transformers=[
            ('num', num_proc, num_features),
            ('num_high', num_proc_high, high_missing_num),
            ('missing_flag', missing_flag_proc,missing_flag_cols),
            ('cat', cat_proc, cat_features),
            #('pos_stats', 'passthrough', pos_stats),
            ('te_cols', 'passthrough', te_cols)
        ],
        remainder='drop'
    )
  return pre
best_params = {'learning_rate': 0.11034679072511581, 'num_leaves': 64, 'max_depth': 12, 'min_child_samples': 69, 'feature_fraction': 0.7189028957238169, 'bagging_fraction': 0.7212431184593704, 'bagging_freq': 6, 'lambda_l1': 2.2629728606140436, 'lambda_l2': 3.208765021602661}

def make_model_pipeline(model,model_name):
  """
  model: LGBMClassifier,CatBoostClassifier, XGBClassifierのインスタンス
  model_name: 'lgbm','cat','xgb'
  """

  preprocessor = make_preprocessor(model_name)

  pipe = Pipeline(steps=[
      ('preprocessor',preprocessor),
      ('model',model)
  ])
  return pipe


In [ ]:
cat = CatBoostClassifier(
    depth=8, learning_rate=0.05, iterations=800,
    loss_function='Logloss', verbose=False
)
pre_tmp = make_preprocessor('cat')
pre_tmp.fit(X,y)

feat_names = pre_tmp.get_feature_names_out()
cat_feature_names = set(cat_features+['School'])

cat_feature_indices = [
    i for i, name in enumerate(feat_names)
    if name.split("__")[-1] in cat_feature_names
]
cat = CatBoostClassifier(
    depth=8, learning_rate=0.05, iterations=800,
    loss_function='Logloss',
    verbose=False,
    cat_features=cat_feature_indices,
)
pre = make_preprocessor('cat')
pipe_cat = Pipeline(steps=[
    ('preprocessor',pre),
    ('model', cat)
])

pipe_cat.fit(X, y)
cat_pred = pipe_cat.predict_proba(X_test)[:,1]

In [ ]:
best_params = {'learning_rate': 0.11034679072511581, 'num_leaves': 64, 'max_depth': 12, 'min_child_samples': 69, 'feature_fraction': 0.7189028957238169, 'bagging_fraction': 0.7212431184593704, 'bagging_freq': 6, 'lambda_l1': 2.2629728606140436, 'lambda_l2': 3.208765021602661}
xgb_preds = []
lgbm_preds = []
seeds = [1,2,3]
for s in seeds:
  lgbm = LGBMClassifier(**best_params,random_state=s,verbose=-1)
  xgb = XGBClassifier(
      n_estimators=800,
      learning_rate=0.02,
      max_depth=12,
      subsample=0.8,
      colsample_bytree=0.8,
      min_child_weight = 5,
      random_state=s
  )
  pipe_xgb = make_model_pipeline(xgb,'xgb')
  pipe_lgbm = make_model_pipeline(lgbm,'lgbm')
  pipe_xgb.fit(X,y)
  pipe_lgbm.fit(X,y)
  xgb_preds.append(pipe_xgb.predict_proba(X_test)[:, 1])
  lgbm_preds.append(pipe_lgbm.predict_proba(X_test)[:,1])



In [ ]:
xgb_pred = np.mean(xgb_preds,axis=0)
lgbm_pred = np.mean(lgbm_preds,axis=0)

In [ ]:

oof_pred = np.zeros(len(X))
pred_proba = np.zeros(len(X_test))

final_pred = 0.6*lgbm_pred + 0.1*xgb_pred + 0.3* cat_pred


In [ ]:
if test_or_sub == 'test':
  oof_cat = np.zeros(len(train))
  for tr_idx,val_idx in CV.split(X,y,groups =group):
    X_trn,y_trn = X.iloc[tr_idx],y.iloc[tr_idx]
    X_val,y_val = X.iloc[val_idx],y.iloc[val_idx]
    pipe_cat.fit(X_trn,y_trn)
    oof_cat[val_idx] = pipe_cat.predict_proba(X_val)[:,1]
    print(f'roc_auc_score:{roc_auc_score(y_val,oof_cat[val_idx])}')
  cat_score = roc_auc_score(y,oof_cat)

roc_auc_score:0.7375770252699418
roc_auc_score:0.7322537112010796
roc_auc_score:0.7521841039846581
roc_auc_score:0.7503117913832198
roc_auc_score:0.7500992063492065
roc_auc_score:0.7617445870314885
roc_auc_score:0.7751971020669081
roc_auc_score:0.7376376163079764
roc_auc_score:0.7221513605442176
roc_auc_score:0.6992063492063493


In [ ]:
if test_or_sub == 'test':
  oof_lgbm = np.zeros(len(train))
  lgbm_scores = []
  for tr_idx,val_idx in CV.split(X,y,groups=group):
    X_trn,y_trn = X.iloc[tr_idx],y.iloc[tr_idx]
    X_val,y_val = X.iloc[val_idx],y.iloc[val_idx]
    pipe_lgbm.fit(X_trn,y_trn)
    oof_lgbm[val_idx] = pipe_lgbm.predict_proba(X_val)[:,1]
    lgbm_scores.append(roc_auc_score(y_val,oof_lgbm[val_idx]))
  for score in lgbm_scores:
    print(f'roc_auc_score:{score}')

roc_auc_score:0.8040731528068291
roc_auc_score:0.812159954542226
roc_auc_score:0.8535549399815328
roc_auc_score:0.8234410430839002
roc_auc_score:0.8378259637188208
roc_auc_score:0.8463734524280626
roc_auc_score:0.8473755238298175
roc_auc_score:0.8518644790112934
roc_auc_score:0.8072420634920635
roc_auc_score:0.7916241496598639


In [ ]:
if test_or_sub == 'test':
  xgb_scores = []
  oof_xgb = np.zeros(len(train))
  for tr_idx,val_idx in CV.split(X,y,groups=group):
    X_trn,y_trn = X.iloc[tr_idx],y.iloc[tr_idx]
    X_val,y_val = X.iloc[val_idx],y.iloc[val_idx]
    pipe_xgb.fit(X_trn,y_trn)
    oof_xgb[val_idx] = pipe_xgb.predict_proba(X_val)[:,1]
    xgb_scores.append(roc_auc_score(y_val,oof_xgb[val_idx]))
  for score in xgb_scores:
    print(f'roc_auc_score:{score}')

roc_auc_score:0.7697439086437899
roc_auc_score:0.7765324241778535
roc_auc_score:0.8359684636692947
roc_auc_score:0.768296485260771
roc_auc_score:0.80687358276644
roc_auc_score:0.8321838430663124
roc_auc_score:0.8165068541799844
roc_auc_score:0.8293912919951703
roc_auc_score:0.7869897959183673
roc_auc_score:0.7671626984126985


In [ ]:
"""
roc_auc_score:0.7966815535078297
roc_auc_score:0.7949002059805383
roc_auc_score:0.8598053839051069
roc_auc_score:0.8015164399092971
roc_auc_score:0.8454790249433107
roc_auc_score:0.8418932670020917
roc_auc_score:0.8328574472618794
roc_auc_score:0.8476738404716244
roc_auc_score:0.7812358276643991
roc_auc_score:0.7905895691609978
"""

'\nroc_auc_score:0.7966815535078297\nroc_auc_score:0.7949002059805383\nroc_auc_score:0.8598053839051069\nroc_auc_score:0.8015164399092971\nroc_auc_score:0.8454790249433107\nroc_auc_score:0.8418932670020917\nroc_auc_score:0.8328574472618794\nroc_auc_score:0.8476738404716244\nroc_auc_score:0.7812358276643991\nroc_auc_score:0.7905895691609978\n'

In [ ]:
Year =train['Year']
CV = StratifiedGroupKFold(n_splits=5,shuffle=True,random_state=1)
if test_or_sub == 'test':
  oof = np.zeros(len(X))
  scores = []
  for trn_idx, val_idx in CV.split(X, y,groups=Year):
      X_tr, X_val = X.iloc[trn_idx], X.iloc[val_idx]
      y_tr, y_val = y[trn_idx], y[val_idx]

      # 各モデルのパイプライン作成
      pipe_lgbm = make_model_pipeline(lgbm,'lgbm')
      pipe_xgb  = make_model_pipeline(xgb,'xgb')
      pipe_cat  = make_model_pipeline(cat,'cat')

      # fit
      pipe_lgbm.fit(X_tr, y_tr)
      pipe_xgb.fit(X_tr, y_tr)
      pipe_cat.fit(X_tr, y_tr)

      # val予測
      pred_l = pipe_lgbm.predict_proba(X_val)[:,1]
      pred_x = pipe_xgb.predict_proba(X_val)[:,1]
      pred_c = pipe_cat.predict_proba(X_val)[:,1]

      # foldごとのアンサンブル
      pred_final = 0.6*pred_l + 0.1*pred_x + 0.3*pred_c

      scores.append(roc_auc_score(y_val,pred_final))
      oof[val_idx] = pred_final

  # CVスコア
  auc = roc_auc_score(y, oof)


In [ ]:
print(scores)
print(auc)

[np.float64(0.8331389139899779), np.float64(0.8804020272584069), np.float64(0.8187783400653078), np.float64(0.8823425570413522), np.float64(0.7886066763425255)]
0.8336872084358381


xgb 0.1 lgbm 0.6 cat 0.3 0.8352649016011714
0.2 0.5 0.3 0.8344692497280718
0.2 0.6 0.2 0.8367365456572606
age missing あり

In [ ]:
lgbm_score = roc_auc_score(y,oof_lgbm)
xgb_score = roc_auc_score(y,oof_xgb)
print(f'lgbm_oof: {lgbm_score}\nxgb_oof:{xgb_score}\ncat_oof:{cat_score}')

lgbm_oof: 0.8282356036916432
xgb_oof:0.8067019634397115
cat_oof:0.7378080386359023


lgbm_oof: 0.828633146074425
xgb_oof:0.8079762540732499
cat_oof:0.7359723115416592
0.8336509135535297

In [ ]:
model_lgbm = pipe_lgbm.named_steps['model']
feat_lgbm = pipe_lgbm.named_steps['preprocessor'].get_feature_names_out()
importances = model_lgbm.feature_importances_
feat_importance_df = pd.DataFrame({
    "feature": feat_lgbm,
    "importance": importances
}).sort_values("importance", ascending=False)

print(feat_importance_df.head(30))

                                     feature  importance
134              te_cols__School_te_smoothed          66
124              pos_stats__SpeedScore_z_pos          55
119              pos_stats__Shuttle_rank_pos          51
109          pos_stats__Sprint_40yd_rank_pos          51
136                te_cols__Year_te_smoothed          46
1                          num__School_count          45
122                  pos_stats__Weight_z_pos          44
16                 missing_flag__Age_missing          42
137  te_cols__Year_Position_Type_te_smoothed          39
108             pos_stats__Sprint_40yd_z_pos          39
120                  pos_stats__Height_z_pos          37
118                 pos_stats__Shuttle_z_pos          37
123               pos_stats__Weight_rank_pos          37
117        pos_stats__Agility_3cone_rank_pos          37
2                                   num__BMI          36
135       te_cols__Position_Type_te_smoothed          35
111        pos_stats__Vertical_

In [ ]:
model_xgb = pipe_xgb.named_steps['model']
feat_xgb = pipe_xgb.named_steps['preprocessor'].get_feature_names_out()
importances = model_xgb.feature_importances_
feat_importance_df = pd.DataFrame({
    "feature": feat_xgb,
    "importance": importances
}).sort_values("importance", ascending=False)

print(feat_importance_df.head(30))

                                               feature  importance
16                           missing_flag__Age_missing    0.233965
75                      cat__Player_Type_special_teams    0.047413
0                                             num__Age    0.046072
40     missing_flag__Broad_Jump_missing_PT_line_backer    0.020166
37   missing_flag__Broad_Jump_missing_PT_offensive_...    0.017931
98                                    cat__Position_RB    0.014080
80                      cat__Position_Type_line_backer    0.013310
70                                      cat__Year_2016    0.012596
51   missing_flag__Shuttle_missing_PT_offensive_lin...    0.012224
104                             cat__missing_cluster_1    0.011885
6                           num_high__Bench_Press_Reps    0.011199
95                                    cat__Position_OT    0.011121
31   missing_flag__Bench_Press_Reps_missing_PT_back...    0.011116
81                cat__Position_Type_offensive_lineman    0.01

In [ ]:
np.corrcoef([oof_lgbm, oof_xgb, oof_cat])

array([[1.        , 0.88219327, 0.61361677],
       [0.88219327, 1.        , 0.53841637],
       [0.61361677, 0.53841637, 1.        ]])

In [ ]:
if True:#test_or_sub == 'sub':
  sub['Drafted'] = final_pred
  sub.to_csv("/content/drive/MyDrive/Colab Notebooks/NFL Draft Prediction/outputs/submission.csv")